<a href="https://colab.research.google.com/github/namtoptall/DataScience/blob/main/rec_sys_milestone_project_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bước 2 : xây dựng hệ thống

In [1]:
# check-gpu
!nvidia-smi

Sun Aug 25 19:24:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# kết nối google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()  # Thực hiện tải tệp kaggle.json lên từ máy tính

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d namdz242221/dsp305x-01-a-vn-dataset
!unzip dsp305x-01-a-vn-dataset.zip -d /content/dataset

In [ ]:
from google.colab import files
files.upload()  # Thực hiện tải ETL_functions.py lên từ máy tính

In [ ]:
from tensorflow.keras.models import load_model

# Đường dẫn đến mô hình đã lưu
fine_tuned_model_path = "/content/drive/MyDrive/Funix_Capstione_model/xception_finetuned_model.keras"

# Tải mô hình đã fine-tune
fine_tuned_model = load_model(fine_tuned_model_path)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Tạo một mô hình chỉ để trích xuất đặc trưng (bỏ đi các lớp đầu ra cuối cùng)
feature_extractor = Model(inputs=fine_tuned_model.input, outputs=fine_tuned_model.get_layer("flatten_2").output)

# Hàm để trích xuất đặc trưng từ hình ảnh
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Chuẩn hóa ảnh
    features = model.predict(img_array)
    return features.flatten()

In [ ]:
# Trích xuất đặc trưng từ tập test
test_dir = "/content/dataset/DSP305x_01-A_VN-dataset/test"  # Thư mục chứa các ảnh trong tập test
feature_list = []
image_list = []

for class_dir in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_dir)
    if os.path.isdir(class_path):
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            if os.path.isfile(img_path):
                features = extract_features(img_path, feature_extractor)
                feature_list.append(features)
                image_list.append(img_path)

feature_array = np.array(feature_list)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_images(input_img_path, feature_array, image_list, model, top_k=5):
    input_features = extract_features(input_img_path, model)
    similarities = cosine_similarity([input_features], feature_array)
    similar_indices = similarities[0].argsort()[-top_k:][::-1]
    recommended_images = [image_list[i] for i in similar_indices]
    return recommended_images

In [ ]:
import matplotlib.pyplot as plt

def show_images(image_paths):
    plt.figure(figsize=(20, 5))
    for i, img_path in enumerate(image_paths):
        img = image.load_img(img_path, target_size=(224, 224))
        plt.subplot(1, len(image_paths), i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

# Thử nghiệm với một ảnh đầu vào
input_image_path = '/path_to_input_image/input_image.jpg'
recommended_images = recommend_images(input_image_path, feature_array, image_list, feature_extractor)
show_images(recommended_images)
